## Imports

In [81]:
import qiskit
from qiskit.quantum_info import states
from qiskit_nature.second_q import hamiltonians, operators
from qiskit_nature.second_q.hamiltonians import lattices 
from qiskit_nature.second_q.hamiltonians.lattices.boundary_condition import BoundaryCondition
from qiskit.circuit import QuantumCircuit
from qiskit.circuit.instruction import Instruction
import numpy as np
from numpy.linalg import eig, norm
import random

## Generate transverse-field Ising operator

currently using the lattice and hamiltonian modules from the qiskit_nature.second_q modules.
I'm not entirely sure why they consider all of this second quantization since we're dealing exclusively with spins, but then again, this is IBM.
They're not exactly known for well-documented or reasonably-organized APIs.

The `HeisenbergModel` module generates the same coupling between each pair in the lattice (on a given vector) and another single-site term.
The transverse-field ising model is a special case of that: the interaction term is ZZ, and the external field is X.

In [24]:
def gen_trans_ising_op(num_qubits: int, zz_coeff: float, x_coeff: float) -> operators.SpinOp:
    lattice = lattices.LineLattice(num_nodes=num_qubits, boundary_condition=BoundaryCondition.OPEN)
    transverse_ham = hamiltonians.HeisenbergModel(lattice=lattice, coupling_constants=(0,0,zz_coeff), ext_magnetic_field=(x_coeff,0,0))
    return transverse_ham.second_q_op()

In [25]:
op = 

## Get exact answers

In [37]:
def get_exact_ground(op: operators.SpinOp) -> tuple[np.float64, np.ndarray]:
    eig_res = eig(op.to_matrix())
    return eig_res.eigenvalues[0].real, eig_res.eigenvectors[0]

In [38]:
print(get_exact_ground(gen_trans_ising_op(4,1,1)))

(2.0941996592125895, array([ 3.57322751e-01+0.j,  1.68874614e-01+0.j,  5.64634443e-01+0.j,
       -9.12210101e-02-0.j, -1.45718492e-01+0.j,  5.59314244e-01+0.j,
        3.89400557e-01+0.j, -8.71088007e-02-0.j, -1.02082419e-01-0.j,
        1.13844761e-01-0.j,  3.19146540e-17+0.j,  1.45417952e-16-0.j,
       -1.31501084e-16+0.j, -1.97452946e-16+0.j,  1.80473133e-17-0.j,
       -9.64668283e-18+0.j]))


## Get expectation value over specific state-vector

In [70]:
def get_expectation_value(state: states.Statevector, op: operators.SpinOp) -> np.float64:
    return np.round(state.expectation_value(op).real, 10)

### Generate example statevector

In [54]:
op = gen_trans_ising_op(4,1,1)
state = states.Statevector.from_instruction(QuantumCircuit(4))
print(get_expectation_value(state, op))

(0.75+0j)


## Generate MUB statevectors
This is done using Dekel's current notebook code.
The code is, for lack of a better word, bad.
I will attempt to prepare it in a cleaner and more robust way.
But for now, let's check that the most basic way still works.

In [64]:
# Adds Y-Hadamard gate on qubit q in circ
def yh(circ, q):
    circ.h(q)
    circ.s(q)

def prep_MUB(circ, A_mat, B_mat, qubits = [0,1]):
    assert len(qubits) == 2
    assert (0 <= A_mat <= 3)
    assert (0 <= B_mat <= 4)
    # A_mat chooses the state in the basis (MUB)
    if A_mat == 1:
        circ.x(qubits[0])
    elif A_mat == 2:
        circ.x(qubits[1])
    elif A_mat == 3:
        circ.x(qubits[0])
        circ.x(qubits[1])
        
    # B_mat chooses the basis (MUB) itself
    if B_mat == 1:
        circ.h(qubits[0])
        circ.h(qubits[1])
    elif B_mat == 2:
        circ.h(qubits[0])
        yh(circ,qubits[1])
        circ.cz(qubits[0], qubits[1])
    elif B_mat == 3:
        yh(circ,qubits[0])
        yh(circ,qubits[1])
    elif B_mat == 4:
        yh(circ,qubits[0])
        circ.h(qubits[1])
        circ.cz(qubits[0], qubits[1])
        
def prep_MUB3(circ, A_mat, B_mat, qubits = [0,1,2]):
    # A_mat chooses the state in the basis (MUB)
    if A_mat == 1:
        circ.x(qubits[0])
    elif A_mat == 2:
        circ.x(qubits[1])
    elif A_mat == 3:
        circ.x(qubits[0])
        circ.x(qubits[1])
    elif A_mat == 4:
        circ.x(qubits[2])
    elif A_mat == 5:
        circ.x(qubits[0])
        circ.x(qubits[2])
    elif A_mat == 6:
        circ.x(qubits[1])
        circ.x(qubits[2])
    elif A_mat == 7:
        circ.x(qubits[0])
        circ.x(qubits[1])
        circ.x(qubits[2])
    
    # B_mat chooses the basis (MUB) itself
    if B_mat == 1:
        circ.h(qubits[0])
        circ.h(qubits[1])
        circ.h(qubits[2])
    elif B_mat == 2:
        yh(circ, qubits[0])
        yh(circ, qubits[1])
        yh(circ, qubits[2])
    elif B_mat == 3:
        yh(circ, qubits[0])
        circ.h(qubits[1])
        circ.h(qubits[2])
        circ.cz(qubits[1], qubits[2])
        circ.cz(qubits[0], qubits[1])
    elif B_mat == 4:
        circ.h(qubits[0])
        yh(circ, qubits[1])
        circ.h(qubits[2])
        circ.cz(qubits[1], qubits[2])
        circ.cz(qubits[0], qubits[2])
    elif B_mat == 5:
        circ.h(qubits[0])
        circ.h(qubits[1])
        yh(circ, qubits[2])
        circ.cz(qubits[0], qubits[1])
        circ.cz(qubits[0], qubits[2])
    elif B_mat == 6:
        yh(circ, qubits[0])
        yh(circ, qubits[1])
        circ.h(qubits[2])
        circ.cz(qubits[0], qubits[1])
        circ.cz(qubits[0], qubits[2])
    elif B_mat == 7:
        yh(circ, qubits[0])
        circ.h(qubits[1])
        yh(circ, qubits[2])
        circ.cz(qubits[1], qubits[2])
        circ.cz(qubits[0], qubits[2])
    elif B_mat == 8:
        circ.h(qubits[0])
        yh(circ, qubits[1])
        yh(circ, qubits[2])
        circ.cz(qubits[0], qubits[1])
        circ.cz(qubits[1], qubits[2])

The field `A_mat` chooses the state *inside* the MUB.  
The field `B_mat` chooses the basis.   
The field `qubits` chooses the subset of qubits over which to generate the MUB state.   

In [62]:
def create_MUB_state(A_mat, B_mat, num_qubits, qubits):
    assert (num_qubits == 2 or num_qubits == 3)
    circuit = QuantumCircuit(num_qubits)
    if num_qubits == 2:
        prep_MUB(circuit, A_mat, B_mat, qubits)
    else:
        prep_MUB3(circuit, A_mat, B_mat, qubits)
    state = states.Statevector.from_instruction(circuit)
    return state

# Attempt all states over a hamiltonian

In [99]:
def try_all_mub_states_textual(op:operators.SpinOp, num_qubits: int=3):
    for i in range(0,5):
        for j in range(0,4):
            state = create_MUB_state(j,i,num_qubits,list(range(num_qubits)))
            value = get_expectation_value(state, op)
            print(f"trying state ({state}), expectation value is {value}")

In [101]:
# MY CODE!
for _ in range(10):
    zz_coeff = random.randint(0,100)
    x_coeff = random.randint(0,100)
    op = gen_trans_ising_op(2, zz_coeff, x_coeff)
    print(f"attempting all MUB states over the operator {op}")
    try_all_mub_states_textual(op, num_qubits=2)


attempting all MUB states over the operator Spin Operator
spin=1/2, number spins=2, number terms=3
  49 * ( Z_0 Z_1 )
+ 48 * ( X_0 )
+ 48 * ( X_1 )
trying state (Statevector([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
            dims=(2, 2))), expectation value is 12.25
trying state (Statevector([0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
            dims=(2, 2))), expectation value is -12.25
trying state (Statevector([0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
            dims=(2, 2))), expectation value is -12.25
trying state (Statevector([0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
            dims=(2, 2))), expectation value is 12.25
trying state (Statevector([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j],
            dims=(2, 2))), expectation value is 48.0
trying state (Statevector([ 0.5+0.j, -0.5+0.j,  0.5+0.j, -0.5+0.j],
            dims=(2, 2))), expectation value is 0.0
trying state (Statevector([ 0.5+0.j,  0.5+0.j, -0.5+0.j, -0.5+0.j],
            dims=(2, 2))), expectation value is -0.0
trying state (Statevector([ 0.5+0.j,

In [86]:
help(random.randint)

Help on method randint in module random:

randint(a, b) method of random.Random instance
    Return random integer in range [a, b], including both end points.



In [71]:
def calculate_energy_landscape(pair_list, qubit_op, offset):
    qubit_mapping_res = []
    for mapping in pair_list:
        MUBs_3qubits = []
        for mat_b in range(9):
            MUBs_3qubits.append([])
            for mat_a in range(8):
                circuit = QuantumCircuit(qubit_op.num_qubits)
                prep_MUB3(circuit, mat_a, mat_b, mapping)
                state = states.Statevector.from_instruction(circuit)
                mub_cost = compute_cost_function(qubit_op, offset, state)
                MUBs_3qubits[-1].append(np.abs(mub_cost))
        qubit_mapping_res.append(MUBs_3qubits)
    return qubit_mapping_res

def flatten_resutls(results):
    flatten_results = []
    for mapping_res in results:
        for MUB_basis in mapping_res:
            for MUB_element in MUB_basis:
                flatten_results.append(MUB_element)
    return flatten_results